In [2]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np
import torch.nn.functional as F

from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, get_svd_encoder
from src.embeddings import RecsysEmbedding
from torch.utils.data import Dataset

experiment_name = 'AttentionGRU2'
device = 'cuda:2'
seed = 123
pkl_path = '../pkl/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
torch.__version__

'1.12.1'

# Модель

In [ ]:
class SlatewiseAttentionLayer(nn.Module):
    """
    Attention-based enoder that operates independently on slates
    """
    def __init__(self, in_dim, nheads=2, out_dim=None):
        super().__init__()
        if out_dim is None:
            out_dim = in_dim
        self.out_dim = out_dim

        self.attention = torch.nn.MultiheadAttention(
            in_dim,
            num_heads=nheads,
            batch_first=True
        ) 
        
        self.lnorm1 = nn.LayerNorm(2 * in_dim)
        self.lnorm2 = nn.LayerNorm(out_dim)

        self.ff = torch.nn.Sequential(
            torch.nn.Linear(2 * in_dim, in_dim),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(in_dim, out_dim),
        )
        
    def forward(self, input, mask):
        # batch, session, slate, embedding
        shp = list(input.shape)
        shp[-1] = self.out_dim

        
        key_padding_mask = mask.clone().flatten(0, 1)
        # let model attent to first padd token if slate is empty 
        key_padding_mask[:, 0] = True
        features = input.flatten(0,1)
        features, attn_map = self.attention(
            features, features, features,
            key_padding_mask=~key_padding_mask
        )
               
        features = torch.cat(
            [features, input.flatten(0,1)],
            dim=-1
        )

        features = self.lnorm1(features)
        features = self.lnorm2(self.ff(features))
        
        return features.reshape(shp)

In [17]:
# torch.autograd.set_detect_anomaly(True)

class AttentionGRU2(torch.nn.Module):
    def __init__(self, embedding, nheads=2, output_dim=1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.attention = torch.nn.MultiheadAttention(
            2 * embedding.embedding_dim,
            num_heads=nheads,
            batch_first=True
        )
        
        self.rnn_layer = torch.nn.GRUCell(
            input_size = embedding.embedding_dim, 
            hidden_size = embedding.embedding_dim, 
        )

        self.hidden_encode_layer = torch.nn.Linear(2 * embedding.embedding_dim, embedding.embedding_dim)
        
        self.out_layer = torch.nn.Sequential(
            torch.nn.Linear(3 * embedding.embedding_dim, embedding.embedding_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(embedding.embedding_dim, output_dim),
        )
    
    
    def get_attention_embeddings(self, item_embs, user_embs, slate_mask):
        shp = item_embs.shape      
        key_padding_mask = slate_mask
        key_padding_mask[:,:, 0] = True # let model attent to first padd token if slate is empty 
        features = torch.cat(
            [
                item_embs,
                user_embs[:, :, None, :].repeat(1, 1, item_embs.size(-2), 1).reshape(shp)
            ],
            dim = -1
        ).flatten(0,1)
        
        features, attn_map = self.attention(
            features, features, features,
            key_padding_mask=~key_padding_mask.flatten(0, 1)
        )
        shp = list(shp)
        shp[-1] *= 2
        features = features.reshape(shp)
        return features
    
    def forward(self, batch):
        # item embs dimensions: batch, sequence, slate, embedding
        # user embs dimensions: batch, sequence, embedding
        item_embs, user_embs = self.embedding(batch)

        # True for real items, False for placeholder
        slate_mask = batch['slates_mask'].clone()

        # item embs dimensions: batch, sequence, slate, embedding
        att_features = self.get_attention_embeddings(item_embs, user_embs, slate_mask)

        # clicked items goes at first 32 coordinates, not clicked at last 32 coordinates
        gru_features, _ = self.rnn_layer(item_embs.flatten(-3, -2))
        gru_features = gru_features.reshape(item_embs.shape)
        
        features = torch.cat(
            [att_features, gru_features],
            dim=-1
        )
        
        return self.out_layer(features).squeeze(-1)


d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = AttentionResponseModel(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
    output_dim=1
).to('cpu')

for batch in dummy_loader:
    print(model(batch))
    break

biulding affinity matrix...


3it [00:00, 5286.94it/s]

tensor([[[-0.0397,  0.0249,  0.0446],
         [ 0.0463,  0.1012,  0.0272]],

        [[-0.1109, -0.1047, -0.1453],
         [-0.0338, -0.0430, -0.0457]]], grad_fn=<SqueezeBackward1>)


# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [21]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = AttentionGRU2(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
    output_dim=1
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 3181.52it/s]


Test before learning: {'f1': 0.4000000059604645, 'roc-auc': 0.3333333134651184, 'accuracy': 0.25}


train... loss:0.6785470247268677:   0%|                                                                                                    | 1/5000 [00:00<39:38,  2.10it/s]

Val update: epoch: 0 |accuracy: 0.25 | f1: 0.4000000059604645 | auc: 1.0 | treshold: 0.01
Test: accuracy: 0.25 | f1: 0.4000000059604645 | auc: 1.0 | 


train... loss:0.6630663275718689:   0%|                                                                                                    | 2/5000 [00:01<39:54,  2.09it/s]

Val update: epoch: 1 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | treshold: 0.52
Test: accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | 


train... loss:0.611418604850769:   0%|                                                                                                     | 6/5000 [00:02<38:05,  2.18it/s]

Val update: epoch: 6 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.5800000000000001
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(AttentionResponseModel(
   (embedding): RecsysEmbedding(
     (item_embeddings): Embedding(5, 32)
   )
   (attention): MultiheadAttention(
     (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
   )
   (rnn_layer): GRU(32, 32, batch_first=True)
   (hidden_encode_layer): Linear(in_features=64, out_features=32, bias=True)
   (out_layer): Sequential(
     (0): Linear(in_features=96, out_features=32, bias=True)
     (1): ReLU()
     (2): Linear(in_features=32, out_features=1, bias=True)
   )
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [22]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [23]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    
    model = AttentionGRU2(
        RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
        output_dim=1
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating SlatewiseAttentionSeqwiseGRU with svd embeddings
Test before learning: {'f1': 0.1819256842136383, 'roc-auc': 0.49667614698410034, 'accuracy': 0.10006504505872726}


train:   0%|                                                                                                                                       | 0/5000 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [7]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [8]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [9]:
for embeddings in ['neural','explicit', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = AttentionGRU2(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings,
            embedding_dim=40
        ),
        output_dim=1
    ).to(device)

    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating InSlateAttentionSequencewiseGRU with neural embeddings
Test before learning: {'f1': 0.7070940136909485, 'roc-auc': 0.5168448090553284, 'accuracy': 0.5949462056159973}


train... loss:58.82067450881004:   0%|                                                                                                  | 1/5000 [01:03<88:29:15, 63.72s/it]

Val update: epoch: 0 |accuracy: 0.7771247625350952 | f1: 0.840562641620636 | auc: 0.861542820930481 | treshold: 0.4
Test: accuracy: 0.7778019309043884 | f1: 0.8418716192245483 | auc: 0.8600804805755615 | 


train... loss:49.33029007911682:   0%|                                                                                                  | 2/5000 [01:11<42:29:25, 30.61s/it]

Val update: epoch: 1 |accuracy: 0.8133314251899719 | f1: 0.8602874875068665 | auc: 0.8926573395729065 | treshold: 0.45
Test: accuracy: 0.8155966401100159 | f1: 0.8628565073013306 | auc: 0.891123354434967 | 


...

train... loss:41.24025613069534:   1%|█                                                                                                 | 54/5000 [06:05<9:13:16,  6.71s/it]

Val update: epoch: 53 |accuracy: 0.841266393661499 | f1: 0.8737034797668457 | auc: 0.916419506072998 | treshold: 0.47000000000000003
Test: accuracy: 0.8391246795654297 | f1: 0.8726430535316467 | auc: 0.9159804582595825 | 


/opt/conda/envs/myenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
train... loss:41.020749509334564:   1%|█                                                                                                | 55/5000 [06:12<9:22:05,  6.82s/it]

Val update: epoch: 54 |accuracy: 0.8394040465354919 | f1: 0.8711230158805847 | auc: 0.9173226952552795 | treshold: 0.47000000000000003
Test: accuracy: 0.8387861251831055 | f1: 0.8711815476417542 | auc: 0.9170501828193665 | 


train... loss:41.69330483675003:   1%|█▏                                                                                                | 62/5000 [07:04<9:23:18,  6.84s/it]


In [10]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items